In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder

# Retrieve data train

In [2]:
knowledgebase = pd.read_excel('https://raw.githubusercontent.com/AndiAlifs/FLUENT-Chatbot-2023/main/KnowledgeBaseFilkom.xlsx', engine='openpyxl')
knowledgebase.head()

,No,Contributor,Topik/Tag,Pertanyaan,Jenis,Jawaban,Link Jawaban,Keterangan Tambahan
0,0.0,Fitra,Informasi Dosen,email Fitra A. Bachtiar,Statis,fitra.bachtiar[at]ub.ac.id,https://filkom.ub.ac.id/sdm-dosen/?search_f7a8...,NaN
1,NaN,NaN,NaN,NIK/NIP Fitra A. Bachtiar,Statis,198406282019031006,NaN,NaN
2,NaN,NaN,NaN,nama lengkap Fitra A. Bachtiar,Statis,Dr.Eng. Fitra A. Bachtiar,https://filkom.ub.ac.id/sdm-dosen/?search_f7a8...,NaN
3,NaN,NaN,NaN,Departemen Fitra A. Bachtiar,Dinamis,Departemen Teknik Informatika,https://filkom.ub.ac.id/sdm-dosen/?search_f7a8...,NaN
4,NaN,NaN,NaN,Program Studi Fitra A. Bachtiar,Dinamis,S2 Ilmu Komputer,https://filkom.ub.ac.id/sdm-dosen/?search_f7a8...,NaN


In [3]:
qa_paired = knowledgebase.drop(columns=knowledgebase.columns.drop(['Pertanyaan', 'Jawaban']))
qa_paired.dropna(inplace=True)
qa_paired

,Pertanyaan,Jawaban
0,email Fitra A. Bachtiar,fitra.bachtiar[at]ub.ac.id
1,NIK/NIP Fitra A. Bachtiar,198406282019031006
2,nama lengkap Fitra A. Bachtiar,Dr.Eng. Fitra A. Bachtiar
3,Departemen Fitra A. Bachtiar,Departemen Teknik Informatika
4,Program Studi Fitra A. Bachtiar,S2 Ilmu Komputer
...,...,...
1229,Apa Manfaat Konseling FILKOM ?,1. Masalah ditangani oleh ahli yang kompeten d...
1230,Berikan informasi mengenai Layanan Konseling,Informasi mengenai Layanan Konseling dapat dia...
1231,Siapa Konselor Bimbingan dan Konseling di FILK...,Ada 2 konselor Bimbingan dan Konseling di FILK...
1232,Siapa Koordinator Konselor Sebaya ?,Koordinator Konselor Sebaya adalah Muhammad Da...


# DNN Based

## Preprocessing data

In [4]:
qa_paired_dnn = qa_paired
qa_paired_dnn['tag'] = qa_paired_dnn.index
qa_paired

,Pertanyaan,Jawaban,tag
0,email Fitra A. Bachtiar,fitra.bachtiar[at]ub.ac.id,0
1,NIK/NIP Fitra A. Bachtiar,198406282019031006,1
2,nama lengkap Fitra A. Bachtiar,Dr.Eng. Fitra A. Bachtiar,2
3,Departemen Fitra A. Bachtiar,Departemen Teknik Informatika,3
4,Program Studi Fitra A. Bachtiar,S2 Ilmu Komputer,4
...,...,...,...
1229,Apa Manfaat Konseling FILKOM ?,1. Masalah ditangani oleh ahli yang kompeten d...,1229
1230,Berikan informasi mengenai Layanan Konseling,Informasi mengenai Layanan Konseling dapat dia...,1230
1231,Siapa Konselor Bimbingan dan Konseling di FILK...,Ada 2 konselor Bimbingan dan Konseling di FILK...,1231
1232,Siapa Koordinator Konselor Sebaya ?,Koordinator Konselor Sebaya adalah Muhammad Da...,1232


In [5]:
X_train = qa_paired_dnn['Pertanyaan'].astype(str)

In [6]:
# tokenize Pertanyaan using tf 

tokenizer = Tokenizer(oov_token='x')
tokenizer.fit_on_texts(X_train)

doc_X = tokenizer.texts_to_sequences(X_train)
doc_X = pad_sequences(doc_X, maxlen=20, padding='post')

docs_Y = qa_paired_dnn['tag']
docs_Y = np.array(docs_Y).reshape(-1, 1)
encoder = OneHotEncoder(sparse=False)
doc_Y = encoder.fit_transform(docs_Y)

output_shape = len(doc_Y[0])

In [261]:
#deep learning model
model = Sequential()
model.add(Dense(8192, input_shape=(20,), activation="relu"))
model.add(Dense(8192, activation="relu"))
model.add(Dense(4096, activation="relu"))
model.add(Dense(2048, activation="relu"))
model.add(Dense(output_shape, activation="sigmoid"))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=["accuracy"])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 8192)              172032    
                                                                 
 dense_31 (Dense)            (None, 8192)              67117056  
                                                                 
 dense_32 (Dense)            (None, 4096)              33558528  
                                                                 
 dense_33 (Dense)            (None, 2048)              8390656   
                                                                 
 dense_34 (Dense)            (None, 1234)              2528466   
                                                                 
Total params: 111,766,738
Trainable params: 111,766,738
Non-trainable params: 0
_________________________________________________________________
None


In [262]:
history_model_dnn = model.fit(x=doc_X, 
                       y=doc_Y,
                       epochs=1500,
                       verbose=1,
                    #    validation_data=(x_test, y_test)
                          )

Epoch 1/1500
39/39 [==============================] - 1s 9ms/step - loss: 29.3403 - accuracy: 0.0000e+00
Epoch 2/1500
39/39 [==============================] - 0s 12ms/step - loss: 7.1308 - accuracy: 0.0000e+00
Epoch 3/1500
39/39 [==============================] - 0s 7ms/step - loss: 6.7316 - accuracy: 8.1037e-04
Epoch 4/1500
39/39 [==============================] - 0s 9ms/step - loss: 6.0685 - accuracy: 0.0041
Epoch 5/1500
39/39 [==============================] - 0s 6ms/step - loss: 5.6069 - accuracy: 0.0081
Epoch 6/1500
39/39 [==============================] - 0s 9ms/step - loss: 5.2496 - accuracy: 0.0219
Epoch 7/1500
39/39 [==============================] - 0s 6ms/step - loss: 4.9876 - accuracy: 0.0284
Epoch 8/1500
39/39 [==============================] - 0s 8ms/step - loss: 4.6516 - accuracy: 0.0405
Epoch 9/1500
39/39 [==============================] - 0s 9ms/step - loss: 4.4309 - accuracy: 0.0511
Epoch 10/1500
39/39 [==============================] - 0s 6ms/step - loss: 4.2293 - ac